In [20]:
import torch
import numpy as np

## Elastic Constitutive Relation

#### Elastic Constitutive Relation Equation
Used to calculate the stress vectors.

\begin{equation} \tag{2}
\sigma = \left\{ \begin{array}{c} 
    \sigma_{xx} \\
    \sigma_{yy} \\
    \tau_{xy}
\end{array} \right\}
= \frac{E}{1 - \nu^2} \left[ \begin{array}{ccc}
    1 & \nu & 0 \\
    \nu & 1 & 0 \\
    0 & 0 & \frac{1 - v}{2}
    \end{array}    
\right] \left\{ \begin{array}{c} 
    \epsilon_{xx} \\
    \epsilon_{yy} \\
    \gamma_{xy}
\end{array} \right\}
\end{equation}

In the case of the inverse problem. Both $E$ and $\nu$ are predicted values. As a result, they themselves are tensors making the calculations more difficult.

In the example code `elastnet.py`, $\nu$ was assumed to be a constant 0.5, which made the matrix multiplication simpler. Below are one or more possible solutions, though they may not be the most optimal.

#### Code provided ($\nu$ = 0.5)

In [24]:
e_xx = torch.tensor(np.array([1., 1., 1.]))
e_yy = torch.tensor(np.array([2., 2., 2.]))
r_xy = torch.tensor(np.array([3., 3., 3.]))
strain= torch.stack([e_xx, e_yy, r_xy], dim = 1)

c_matrix = torch.tensor(
    (1/(1-(1/2.0)**2))*np.array([[1, 1/2.0, 0], [1/2.0, 1, 0], [0, 0, (1-1/2.0)/2.0]])
)

pred_E = torch.tensor([0.5, 1.0, 1.5])
modulus = torch.stack([pred_E, pred_E, pred_E], axis = 1)

stress = torch.multiply(torch.matmul(strain, c_matrix), modulus)
stress

tensor([[1.3333, 1.6667, 0.5000],
        [2.6667, 3.3333, 1.0000],
        [4.0000, 5.0000, 1.5000]], dtype=torch.float64)

#### Calculating each value individually testing

In [33]:
e_xx = torch.tensor(np.array([1., 1., 1.]))
e_yy = torch.tensor(np.array([2., 2., 2.]))
r_xy = torch.tensor(np.array([3., 3., 3.]))
strain= torch.stack([e_xx, e_yy, r_xy], dim = 1)

c_matrix = torch.tensor(
    (1/(1-(1/2.0)**2))*np.array([[1, 1/2.0, 0], [1/2.0, 1, 0], [0, 0, (1-1/2.0)/2.0]])
)

pred_E = torch.tensor([0.5, 1.0, 1.5])
modulus = torch.stack([pred_E, pred_E, pred_E], axis = 1)

stress = torch.multiply(torch.matmul(strain, c_matrix), modulus)

v_dat = torch.tensor(np.array([0.5, 0.5, 0.5]))
v_stack = torch.stack([v_dat, v_dat, v_dat])

c_mat = torch.tensor(np.array([
    torch.tensor(np.array([
        [1.,  v*1.0,  0.],
        [v * 1.0,  1.,  0.],
        [0., 0., (1.0-v)/2.0]
    ])) for v in v_stack[:,0]
]))

v2 = torch.multiply(v_stack, v_stack)

fraction = torch.divide(modulus, 1 - v2)

mat_res = torch.matmul(strain, c_mat)

result = torch.multiply(fraction, mat_res)

result

tensor([[[1.3333, 1.6667, 0.5000],
         [2.6667, 3.3333, 1.0000],
         [4.0000, 5.0000, 1.5000]],

        [[1.3333, 1.6667, 0.5000],
         [2.6667, 3.3333, 1.0000],
         [4.0000, 5.0000, 1.5000]],

        [[1.3333, 1.6667, 0.5000],
         [2.6667, 3.3333, 1.0000],
         [4.0000, 5.0000, 1.5000]]], dtype=torch.float64)